<a href="https://colab.research.google.com/github/zrghassabi/LLM/blob/main/Chapter5_Demo_Chatbot_Flask_App_with_Sentiment_Summarization_QA%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Chatbot with Sentiment Analysis, Summarization, and Q&A Using Flask

This notebook sets up a Flask application for a chatbot that integrates three different models: sentiment analysis, summarization, and question answering. It also includes a fallback decoder-only model. The chatbot works as follows:
- When asked a question, it uses the Q&A model.
- On every message, it checks the sentiment using the sentiment analysis model and adjusts the prompt to the decoder model accordingly.
- When asked to summarize something, it uses the summarization model.
- All messages are saved so the decoder model has more context.
- There are endpoints for resetting the conversation and greeting the user.


In [ ]:
!pip install transformers flask requests tensorflow


### Flask App Initialization

We initialize the Flask application and set up global variables to store conversation history and conversation summaries.



### Helper Functions

We define helper functions to:
- Summarize the conversation.
- Summarize a given text.
- Answer questions.
- Generate responses using the decoder model.
- Analyze sentiment.



### Endpoints

We define three endpoints:
- `/reset`: Resets the conversation history after summarizing it.
- `/greet`: Provides a greeting message.
- `/chat`: Processes user messages, checks sentiment, and uses the appropriate model to generate responses.


In [ ]:
%%writefile chatbot.py

# Step 1: Load the necessary libraries and models
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline, TFAutoModelForSeq2SeqLM
from flask import Flask, request, jsonify
import json

# Load Sentiment Analysis Model
sentiment_model = TFAutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
sentiment_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Load Summarization Model
summarization_model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
summarization_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Load Question Answering Model
qa_model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Load Generic Decoder-Only Model
pipe = pipeline("text-generation", model="gpt2-large")


# Initialize Flask App
app = Flask(__name__)
conversation_history = []
conversation_summaries = []


def summarize_conversation(conversation):
    conversation_text = " ".join(conversation)
    inputs = summarization_tokenizer(f'Summarize: {conversation_text}', return_tensors="tf", max_length=512, truncation=True, padding=True)
    summary_ids = summarization_model.generate(inputs['input_ids'], max_length=500, num_beams=4, early_stopping=True)
    summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def summarize_text(text):
    inputs = summarization_tokenizer(f'Summarize: {text}', return_tensors="tf", max_length=512, truncation=True, padding=True)
    summary_ids = summarization_model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
    summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def answer_question(question):
    inputs = qa_tokenizer(f'question: {question} answer: ', return_tensors="tf")
    answer_ids = qa_model.generate(inputs['input_ids'])
    answer = qa_tokenizer.decode(answer_ids[0], skip_special_tokens=True)
    return answer

def generate_response(message):
    # context = " ".join(conversation_history)
    sentiment = analyze_sentiment(message)
    if sentiment == "negative":
        prompt = f"The user is angry. Their message: {message}"
    else:
        prompt = message
    response = pipe(prompt)[0]['generated_text']
    return response

def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="tf", truncation=True)
    outputs = sentiment_model(inputs)
    sentiment_score = tf.nn.softmax(outputs.logits, axis=-1).numpy()
    sentiment = "positive" if sentiment_score[0][1] > 0.5 else "negative"
    return sentiment


@app.route('/reset', methods=['POST'])
def reset_conversation():
    global conversation_history
    global conversation_summaries
    summary = summarize_conversation(conversation_history)
    conversation_summaries.append(summary)
    conversation_history = []
    return jsonify({"message": "Conversation has been reset.", "summary": summary})

@app.route('/greet', methods=['GET'])
def greet_user():
    return jsonify({"message": "Hello! How can I assist you today?"})

@app.route('/chat', methods=['POST'])
def chat():
    global conversation_history
    user_message = request.json.get('message')
    conversation_history.append(f'User: {user_message}')

    # Determine the task based on the message
    if "summarize" in user_message.lower():
        print("summarize")
        response = summarize_text(user_message)
    elif "?" in user_message:
        print("question")
        response = answer_question(user_message)
    else:
        print("default")
        response = generate_response(user_message)

    sentiment = analyze_sentiment(user_message)
    conversation_history.append(f'Agent: {response}')

    return jsonify({"response": response})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


### Running the Flask App

Finally, we run the Flask app.


In [ ]:
!nohup python chatbot.py &


In [ ]:
!sudo lsof -i -P -n | grep LISTEN

In [ ]:
from requests import post, get
from socket import gethostname, gethostbyname
ip = gethostbyname(gethostname()) # 172.28.0.12
response = get(f"http://{ip}:5000/greet").json()
response

In [ ]:
response = post(f"http://{ip}:5000/chat", json= {'message': 'What is the capital of France?'}).json()
response

In [ ]:
response = post(f"http://{ip}:5000/chat", json= {'message': 'summarize this text \n The Bastille was a fortress in Paris, known as the Bastille Saint-Antoine. It played an important role in the internal conflicts of France and for most of its history was used as a state prison by the kings of France. It was stormed by a crowd on 14 July 1789, in the French Revolution, becoming an important symbol for the French Republican movement. It was later demolished and replaced by the Place de la Bastille.'}).json()
response

In [ ]:
response = post(f"http://{ip}:5000/chat", json= {'message': 'I want to build an app'}).json()
response

In [ ]:
response = post(f"http://{ip}:5000/reset").json()
response